<a href="https://colab.research.google.com/github/theAyushAT/small-projects/blob/main/Object-detetection-on-aerial-images/object_detection_satellite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#GPU count and name
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1ba4eb95-86ac-f309-d5cc-5f04a74b83e6)


In [ ]:
!pip install Keras==2.3.1

     |████████████████████████████████| 378kB 8.4MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [ ]:
!python -c 'import keras; print(keras.__version__)'

Using TensorFlow backend.
2021-03-27 02:44:22.967511: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2.3.1


## Download and extract dataset

In [ ]:
import os

# replace with your own Kaggle username and key

os.environ['KAGGLE_USERNAME'] = "theayushtiwari" 
os.environ['KAGGLE_KEY'] = "583a98462d37f5eec9ed8bc96d718c1" 

#data link https://www.kaggle.com/kbhartiya83/swimming-pool-and-car-detection
!kaggle datasets download kbhartiya83/swimming-pool-and-car-detection -p /content/data --unzip 

 91% 101M/111M [00:01<00:00, 74.1MB/s] 
100% 111M/111M [00:01<00:00, 72.1MB/s]


## Clone Retinanet Repo

In [ ]:
import os

repo_url = 'https://github.com/fizyr/keras-retinanet'
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url))) 

!git clone {repo_url} # clone repo

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 6205, done.
remote: Total 6205 (delta 0), reused 0 (delta 0), pack-reused 6205
Receiving objects: 100% (6205/6205), 13.47 MiB | 32.83 MiB/s, done.
Resolving deltas: 100% (4201/4201), done.


In [ ]:
# from /content/keras-retinanet
%cd {repo_dir_path} 
!pip install .
!python setup.py build_ext --inplace

%cd ..

/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-1.0.0-cp37-cp37m-linux_x86_64.whl size=168007 sha256=42ff0d740bc790d2f96e6f94af1055e7fcfe3c621c4d27947792f11873bcfcd2
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20486 sha256=a115fa19bf6e8d5271bd2156e55e4dd72ff6917cac6649b519c4fab091e28de9
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
Successfully built keras-retinanet keras-resnet
running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/keras-retinanet/k

## Clone ije_retinanet Repo

In [ ]:
ije_repo_url = 'https://github.com/ijeism/ije_retinanet'
ije_repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(ije_repo_url))) 

!git clone {ije_repo_url} # clone repo
# %cd {ije_repo_dir_path} 
# !git pull # pull repo

Cloning into 'ije_retinanet'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 78 (delta 39), reused 41 (delta 15), pack-reused 0
Unpacking objects: 100% (78/78), done.


## Prepare dataset for training

In [ ]:
import os
base_dir = os.getcwd()
base_dir

'/content'

In [ ]:
# select subsample of N for initial training

from shutil import copyfile
import random

# determine number for subsample
NUM = 800

# create dir for subsample
sub_dir = base_dir + '/images_subset/'
!mkdir {sub_dir}

image_dir = base_dir + '/data/training_data/training_data/images/'
image_paths = os.listdir(image_dir)
random_NUM = random.sample(image_paths, NUM) # randomly select subsample 

# copy subsample into subsample dir
for i in random_NUM:
    copyfile(image_dir + i, sub_dir + i )

In [ ]:
%cd /content/images_subset/
!ls -F |grep -v / | wc -l

/content/images_subset
800


In [ ]:
!python /content/ije_retinanet/build_dataset.py \
-l /content/data/training_data/training_data/labels/ \
-i /content/images_subset/ \
-r /content/images_subset/train.csv \
-e /content/images_subset/test.csv \
-c /content/images_subset/classes.csv 

[INFO] creating 'train' set ...
[INFO] 640 total images in 'train'
[INFO] writing train annotations ...
[INFO] total 2468 annotations
[INFO] train.csv completed
[INFO] creating 'test' set ...
[INFO] 160 total images in 'test'
[INFO] writing test annotations ...
[INFO] total 923 annotations
[INFO] test.csv completed
[INFO] writing classes ...
[INFO] classes.csv completed
[FINAL] Task completed!


## Download Pretrained Model

In [ ]:
import urllib.request

PRETRAINED_MODEL = '/content/keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5'

URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'

urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

('/content/keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5',
 <http.client.HTTPMessage at 0x7f91ea5a1650>)

## Train CNN

In [ ]:
!wc -l /content/images_subset/train.csv

2468 /content/images_subset/train.csv


In [ ]:
# compute number of annotations to compute number of steps per epoch

import pandas as pd
import math

df = pd.read_csv('/content/images_subset/train.csv', header=None)
count = len(df)
no_steps = math.ceil(count/32)

print("Count of annotations: {}".format(count))
print("Number of steps per epoch: {}".format(no_steps))

Count of annotations: 2468
Number of steps per epoch: 78


In [ ]:
!pip install tensorflow --upgrade

In [ ]:
!mkdir /content/run_1/
%cd /content/run_1/

!python /content/keras-retinanet/keras_retinanet/bin/train.py \
--freeze-backbone \
--random-transform \
--weights {PRETRAINED_MODEL} \
--weighted-average \
--batch-size 8 \
--steps {no_steps} \
--epochs 100 \
csv '/content/images_subset/train.csv' '/content/images_subset/classes.csv'

mkdir: cannot create directory ‘/content/run_1/’: File exists
/content/run_1
2021-03-27 03:01:39.959644: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Creating model, this may take a second...
2021-03-27 03:01:41.613733: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-27 03:01:41.614530: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-27 03:01:41.650920: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-27 03:01:41.651484: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB

## Evaluate model

In [ ]:
# path to trained weights
model_1_path = os.path.join('/content/run_1/snapshots/', sorted(os.listdir('/content/run_1/snapshots/'), reverse=True)[0])
print(model_1_path)

# evaluate model on test.csv
!python /content/keras-retinanet/keras_retinanet/bin/evaluate.py \
csv '/content/images_subset/test.csv' \
'/content/images_subset/classes.csv' \
{model_1_path} --convert-model

%cd ..

/content/run_1/snapshots/resnet50_csv_79.h5
2021-03-27 04:51:47.225458: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Loading model, this may take a second...
2021-03-27 04:51:54.434595: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-27 04:51:54.446522: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-27 04:51:54.524459: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-27 04:51:54.525084: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/

## Generate detections on test images - ANNOTATION FILES

In [ ]:
# create directory with selected test images 

!mkdir /content/data/test_data_images/test_data_images/test
!mv -t /content/data/test_data_images/test_data_images/test \
/content/data/test_data_images/test_data_images/images/000000091.jpg \
/content/data/test_data_images/test_data_images/images/000000097.jpg \
/content/data/test_data_images/test_data_images/images/000002576.jpg \
/content/data/test_data_images/test_data_images/images/000002412.jpg \
/content/data/test_data_images/test_data_images/images/000000076.jpg \
/content/data/test_data_images/test_data_images/images/000000441.jpg

## Convert training model to inference model

In [ ]:
# print trained model path
model_1_path

'/content/run_1/snapshots/resnet50_csv_79.h5'

In [ ]:
os.getcwd()

'/content'

In [ ]:
# convert trained model to inference model to be used for generating predictions
!python /content/keras-retinanet/keras_retinanet/bin/convert_model.py '/content/run_1/snapshots/resnet50_csv_30.h5' '/content/run_1/snapshots/resnet50_csv_30_inference.h5'
inference_model = '/content/run_1/snapshots/resnet50_csv_30_inference.h5'

2021-03-27 04:53:27.174122: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-27 04:53:28.729174: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-27 04:53:28.730001: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-27 04:53:28.764380: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-27 04:53:28.765002: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-03-27 04:53:28.765048: I tensorflow/stream_executor/platform/default/dso_loade

In [ ]:
# generate prediction files

!python /content/ije_retinanet/image_inference_write.py \
-i '/content/data/test_data_images/test_data_images/test/' \
-t 0.6 \
-m {inference_model} \
-o  /content/data/ \
-l /content/images_subset/classes.csv

2021-03-27 04:53:40.473128: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-27 04:53:42.750471: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-27 04:53:42.751271: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-27 04:53:42.780027: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-27 04:53:42.780626: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-03-27 04:53:42.780673: I tensorflow/stream_executor/platform/default/dso_loade

In [ ]:
# combine all csv files into one
import glob
fout=open("/content/data/output/out.csv","a")
fout.write("path,confidence,ymin,xmin,ymax,xmax,class\n") # create header
for fi in glob.glob("/content/data/*.csv"):
  f = open(fi)
  for line in f:
    fout.write(line)
fout.close()

## Generate detections on test images - IMAGES

In [ ]:
os.getcwd()

'/content'

In [ ]:
# create output directory where you want to save images with bounding boxes
!mkdir /content/data/output

# generate detections on images
!python /content/ije_retinanet/image_inference_print.py \
-i /content/data/test_data_images/test_data_images/test/ \
-t 0.6 \
-m  {inference_model} \
-o /content/data/output

mkdir: cannot create directory ‘/content/data/output’: File exists
2021-03-27 05:04:41.472737: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-27 05:04:43.564585: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-27 05:04:43.565462: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-27 05:04:43.598388: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-27 05:04:43.599015: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-03-27 05:04

In [ ]:
from google.colab import files

# download output images
for p in glob.glob('/content/data/output/*.jpg'):
  files.download(p)
  
# download raw images
for p in glob.glob('/content/data/test_data_images/test_data_images/test/*.jpg'):
  files.download(p)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# create a dataframe for analysis purposes

import pandas as pd
df = pd.read_csv("/content/data/output/out.csv")
df.head()

,path,confidence,ymin,xmin,ymax,xmax,class
0,path,confidence,ymin,xmin,ymax,xmax,class
1,/content/data/test_data_images/test_data_image...,0.6900253,204.23952,110.533905,215.65071,122.34009,1
2,/content/data/test_data_images/test_data_image...,0.80099934,52.44447,61.93716,63.514984,74.036766,1
3,/content/data/test_data_images/test_data_image...,0.7431664,86.74582,173.33273,98.36514,184.52744,1
4,/content/data/test_data_images/test_data_image...,0.6905191,188.9087,85.26168,223.72,134.94113,2


In [ ]:
df["class"].value_counts()

1        18
2         3
class     1
Name: class, dtype: int64